In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

In [2]:
training_data = MNIST(root="data", train=True, download=True, transform=ToTensor())
test_data = MNIST(root="data", train=False, download=True, transform=ToTensor())
print(len(training_data), len(test_data))

60000 10000


In [3]:
batch_size = 64

# create data loaders
training_loader = DataLoader(training_data, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=64)

for X, y in test_loader:
    print(f"Shape of X [N C H W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N C H W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [4]:
# get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    print(f"size: {size}")
    for batch, (X, y) in enumerate(dataloader):
        X = X.to(device)  # Move input data to the device (GPU or CPU)
        y = y.to(device)  # Move target labels to the device (GPU or CPU)

        # compute predicted y by passing X to the model
        prediction = model(X)        # compute the loss
        loss = loss_fn(prediction, y)      #  apply zero gradients, perform a backward pass, and update the weights
        optimizer.zero_grad()  
        loss.backward()  
        optimizer.step()          # print training progress
        if batch % 100 == 0:
            loss_value = loss.item()  
            current = batch * len(X)
            print(f"loss: {loss_value:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            prediction = model(X)
            test_loss += loss_fn(prediction, y).item()
            correct += (prediction.argmax(1) == y).type(torch.float).sum().item()
        test_loss /= num_batches
        correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epoch = 5
for t in range(epoch):
  print(f"Epoch {t+1}\n-------------------------------")
  train(training_loader, model, loss_fn, optimizer)
  test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
size: 60000
loss: 2.301475  [    0/60000]
loss: 2.273239  [ 6400/60000]
loss: 2.256021  [12800/60000]
loss: 2.127163  [19200/60000]
loss: 2.103697  [25600/60000]
loss: 2.026656  [32000/60000]
loss: 1.704990  [38400/60000]
loss: 1.589985  [44800/60000]
loss: 1.251858  [51200/60000]
loss: 1.202046  [57600/60000]
Test Error: 
 Accuracy: 69.3%, Avg loss: 1.108224 

Epoch 2
-------------------------------
size: 60000
loss: 1.208255  [    0/60000]
loss: 1.085050  [ 6400/60000]
loss: 1.005336  [12800/60000]
loss: 0.738770  [19200/60000]
loss: 0.740419  [25600/60000]
loss: 0.793943  [32000/60000]
loss: 0.812519  [38400/60000]
loss: 0.785065  [44800/60000]
loss: 0.664968  [51200/60000]
loss: 0.698157  [57600/60000]
Test Error: 
 Accuracy: 88.1%, Avg loss: 0.514717 

Epoch 3
-------------------------------
size: 60000
loss: 0.562591  [    0/60000]
loss: 0.447395  [ 6400/60000]
loss: 0.430890  [12800/60000]
loss: 0.460386  [19200/60000]
loss: 0.379351  [256

In [13]:
classes = [
    "0", "1", "2", "3", "4",
    "5", "6", "7", "8", "9"
]

model.eval()

sample_index = 1
x, y = test_data[sample_index][0], test_data[sample_index][1]

with torch.no_grad():
    x = x.to(device)
    prediction = model(x.unsqueeze(0))

    predicted, actual = classes[prediction.argmax(dim=1)], classes[y]
    print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: '2', Actual: '2'
